In [61]:
#import library
import requests
import pandas as pd
import numpy as np
import pandas.util.testing as tm

#setting directory where python interpreter looks for code files
import sys
sys.path.insert(0, './scripts')

#read in functions from script file
from codes import data_exploration, calc_out_weights, calc_in_players_weights

ImportError: cannot import name 'calc_in_players_weights' from 'codes' (./scripts/codes.py)

Data Ingestion Phase

In [2]:
# identify the available season historical data available through fantasy Nutmeg website
r= requests.get('https://www.fantasynutmeg.com/api/history/season')
df = pd.DataFrame(r.json())
print(df)

         0
0  2016-17
1  2017-18
2  2018-19
3  2019-20
4  2020-21
5  2021-22
6  2022-23


In [3]:
# get yearly historic data from endpoint for available seasons and identify the keys in each disctionary using 2016 as an example
Y2016= requests.get('https://www.fantasynutmeg.com/api/history/season/2016-17').json()
Y2017= requests.get('https://www.fantasynutmeg.com/api/history/season/2017-18').json()
Y2018= requests.get('https://www.fantasynutmeg.com/api/history/season/2018-19').json()
Y2019= requests.get('https://www.fantasynutmeg.com/api/history/season/2019-20').json()
Y2020= requests.get('https://www.fantasynutmeg.com/api/history/season/2020-21').json()
Y2021= requests.get('https://www.fantasynutmeg.com/api/history/season/2021-22').json()
Y2022= requests.get('https://www.fantasynutmeg.com/api/history/season/2022-23').json()
Y2016.keys()

dict_keys(['dd_agg_fixture', 'dd_agg_player', 'dd_hauls', 'history'])

In [4]:
#create dataframes for all the available seasons dictionary keys for data exploration
dd_agg16_df = pd.DataFrame(Y2016['dd_agg_fixture'])
dd_hauls16_df = pd.DataFrame(Y2016['dd_hauls'])
dd_agg_pl16_df = pd.DataFrame(Y2016['dd_agg_player'])
hist16_df = pd.DataFrame(Y2016['history'])
dd_agg17_df = pd.DataFrame(Y2017['dd_agg_fixture'])
dd_hauls17_df = pd.DataFrame(Y2017['dd_hauls'])
dd_agg_pl17_df = pd.DataFrame(Y2017['dd_agg_player'])
hist17_df = pd.DataFrame(Y2017['history'])
dd_agg18_df = pd.DataFrame(Y2018['dd_agg_fixture'])
dd_hauls18_df = pd.DataFrame(Y2018['dd_hauls'])
dd_agg_pl18_df = pd.DataFrame(Y2018['dd_agg_player'])
hist18_df = pd.DataFrame(Y2018['history'])
dd_agg19_df = pd.DataFrame(Y2019['dd_agg_fixture'])
dd_hauls19_df = pd.DataFrame(Y2019['dd_hauls'])
dd_agg_pl19_df = pd.DataFrame(Y2019['dd_agg_player'])
hist19_df = pd.DataFrame(Y2019['history'])
dd_agg20_df = pd.DataFrame(Y2020['dd_agg_fixture'])
dd_hauls20_df = pd.DataFrame(Y2020['dd_hauls'])
dd_agg_pl20_df = pd.DataFrame(Y2020['dd_agg_player'])
hist20_df = pd.DataFrame(Y2020['history'])
dd_agg21_df = pd.DataFrame(Y2021['dd_agg_fixture'])
dd_hauls21_df = pd.DataFrame(Y2021['dd_hauls'])
dd_agg_pl21_df = pd.DataFrame(Y2021['dd_agg_player'])
hist21_df = pd.DataFrame(Y2021['history'])
dd_agg22_df = pd.DataFrame(Y2022['dd_agg_fixture'])
dd_hauls22_df = pd.DataFrame(Y2022['dd_hauls'])
dd_agg_pl22_df = pd.DataFrame(Y2022['dd_agg_player'])
hist22_df = pd.DataFrame(Y2022['history'])

In [5]:
#get current season data from FPL API endpoints and identify the keys
fpl_base_url = 'https://fantasy.premierleague.com/api/'
current_season = requests.get(fpl_base_url+'bootstrap-static/').json()
#json = r.json()
current_season.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [6]:
#create dataframes for the current season dictionary keys for data exploration
#- Contains summary of Gameweek data
events_df = pd.DataFrame(current_season['events']) #
phases_df = pd.DataFrame(current_season['phases']) #Shows calendar months for game weeks
teams_df = pd.DataFrame(current_season['teams'])
players_df = pd.DataFrame(current_season['elements'])
element_stats_df = pd.DataFrame(current_season['element_stats'])
element_types_df = pd.DataFrame(current_season['element_types'])

In [7]:
#get current season fixtures from FPL API endpoint and create Dataframe

current_season_fixtures = requests.get(fpl_base_url+'fixtures/').json()
fixtures_df = pd.DataFrame(current_season_fixtures)
fixtures_df.head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,2292810,1,True,True,1,2022-08-05T19:00:00Z,90,False,True,1,2.0,7,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,2,74911
1,2292813,1,True,True,4,2022-08-06T11:30:00Z,90,False,True,12,2.0,9,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,74914
2,2292811,1,True,True,2,2022-08-06T14:00:00Z,90,False,True,2,0.0,3,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,2,74912
3,2292814,1,True,True,5,2022-08-06T14:00:00Z,90,False,True,20,1.0,11,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,74915
4,2292816,1,True,True,7,2022-08-06T14:00:00Z,90,False,True,16,0.0,15,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,3,74917


Data Exploration Phase

In [8]:
#explore the historical double digit aggregate fixture dataframe using Y2016 data
data_exploration(dd_agg16_df)


 There are 40 rows and 2 columns

The columns in the dataset are
Index(['count', 'fixture'], dtype='object')

The columns with null values in the dataset are
[]

The data types and null values count in the dataset are
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   count    40 non-null     int64 
 1   fixture  40 non-null     object
dtypes: int64(1), object(1)
memory usage: 768.0+ bytes
None


In [9]:
#explore the historical double digit aggregate fixture dataframe using Y2016 data
dd_agg16_df.head()

,count,fixture
0,22,HUL(H)
1,22,WAT(H)
2,19,LEI(H)
3,19,SWA(A)
4,18,SUN(A)


Observation: The double digit aggregate dataframe explored above contains the fixtures and the counts of players with a haul (double digits) fantasy points in each fixture.

In [10]:
#explore the season history data for the players
data_exploration(hist16_df)


 There are 683 rows and 59 columns

The columns in the dataset are
Index(['assists', 'bonus', 'bps', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'clean_sheets', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'creativity', 'dreamteam_count', 'ea_index',
       'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name',
       'form', 'goals_conceded', 'goals_scored', 'ict_index', 'id',
       'in_dreamteam', 'influence', 'loaned_in', 'loaned_out', 'loans_in',
       'loans_out', 'minutes', 'news', 'now_cost', 'own_goals',
       'penalties_missed', 'penalties_saved', 'photo', 'points_per_game',
       'position', 'red_cards', 'saves', 'second_name', 'selected_by_percent',
       'special', 'squad_number', 'status', 'team', 'team_code', 'team_name',
       'threat', 'total_points', 'transfers_in', 'transfers_in_event',
       'transfers_out', 'transfers_out_event', 'value_form', 'valu

In [11]:
#explore the season history data for the players
hist16_df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,...,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
0,0,0,18,100,100,0,48844,0,0,-3,...,0.0,2,3433,0,7905,0,0.0,0.4,Ospina,0
1,0,2,660,100,100,12,11334,0,0,-1,...,0.0,134,432837,0,590727,0,0.0,24.8,Cech,2
2,1,19,723,0,75,10,51507,0,0,1,...,239.0,121,965040,0,857277,0,0.0,19.8,Koscielny,4
3,0,0,5,100,100,0,17127,0,0,-2,...,0.0,1,2141,0,5912,0,0.0,0.2,Mertesacker,0
4,0,2,296,75,100,5,158074,0,0,-2,...,119.0,45,119600,0,100361,0,0.0,9.4,Gabriel,6


Observation: The season history dataframe explored above contains the season performance data for each player.

In [12]:
#explore the historical double digit aggregate for each player dataframe
data_exploration(dd_agg_pl16_df)


 There are 206 rows and 2 columns

The columns in the dataset are
Index(['count', 'player'], dtype='object')

The columns with null values in the dataset are
[]

The data types and null values count in the dataset are
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   count   206 non-null    int64 
 1   player  206 non-null    object
dtypes: int64(1), object(1)
memory usage: 3.3+ KB
None


In [13]:
#explore the historical double digit aggregate for each player dataframe
dd_agg_pl16_df.head()

,count,player
0,2,Defour
1,7,Coutinho
2,4,Lallana
3,4,Milner
4,5,Daniels


Observation: The players double digit aggregate dataframe explored above contains the count of double digits points games by each player in the season.

In [14]:
#explore the historical double digits hauls dataframe
data_exploration(dd_hauls16_df)


 There are 469 rows and 5 columns

The columns in the dataset are
Index(['fixture', 'player', 'points', 'position', 'round'], dtype='object')

The columns with null values in the dataset are
[]

The data types and null values count in the dataset are
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469 entries, 0 to 468
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   fixture   469 non-null    object
 1   player    469 non-null    object
 2   points    469 non-null    int64 
 3   position  469 non-null    object
 4   round     469 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 18.4+ KB
None


In [15]:
#explore the historical double digits hauls dataframe
dd_hauls16_df.head()

,fixture,player,points,position,round
0,HUL(H),Defour,11,MID,4
1,HUL(H),Coutinho,14,MID,6
2,HUL(H),Lallana,15,MID,6
3,HUL(H),Milner,14,MID,6
4,HUL(H),Daniels,13,DEF,8


Observation: The double digits haul dataframe explored above contains the double digits points by players in different gameweek fixtures in the season.

In [16]:
#explore the current season events dataframe
data_exploration(events_df)


 There are 38 rows and 23 columns

The columns in the dataset are
Index(['id', 'name', 'deadline_time', 'average_entry_score', 'finished',
       'data_checked', 'highest_scoring_entry', 'deadline_time_epoch',
       'deadline_time_game_offset', 'highest_score', 'is_previous',
       'is_current', 'is_next', 'cup_leagues_created',
       'h2h_ko_matches_created', 'chip_plays', 'most_selected',
       'most_transferred_in', 'top_element', 'top_element_info',
       'transfers_made', 'most_captained', 'most_vice_captained'],
      dtype='object')

The columns with null values in the dataset are
['highest_scoring_entry', 'highest_score', 'most_selected', 'most_transferred_in', 'top_element', 'top_element_info', 'most_captained', 'most_vice_captained']

The data types and null values count in the dataset are
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------             

In [17]:
#explore the current season events dataframe
events_df.head()

,id,name,deadline_time,average_entry_score,finished,data_checked,highest_scoring_entry,deadline_time_epoch,deadline_time_game_offset,highest_score,...,cup_leagues_created,h2h_ko_matches_created,chip_plays,most_selected,most_transferred_in,top_element,top_element_info,transfers_made,most_captained,most_vice_captained
0,1,Gameweek 1,2022-08-05T17:30:00Z,57,True,True,7309963.0,1659720600,0,119.0,...,False,False,"[{'chip_name': 'bboost', 'num_played': 170273}...",28.0,1.0,366.0,"{'id': 366, 'points': 15}",0,283.0,318.0
1,2,Gameweek 2,2022-08-13T10:00:00Z,57,True,True,3977474.0,1660384800,0,139.0,...,False,False,"[{'chip_name': 'bboost', 'num_played': 136909}...",28.0,318.0,28.0,"{'id': 28, 'points': 19}",12847501,318.0,283.0
2,3,Gameweek 3,2022-08-20T10:00:00Z,48,True,True,9105261.0,1660989600,0,118.0,...,False,False,"[{'chip_name': 'bboost', 'num_played': 118152}...",28.0,19.0,7.0,"{'id': 7, 'points': 16}",20175060,28.0,318.0
3,4,Gameweek 4,2022-08-27T10:00:00Z,62,True,True,9406675.0,1661594400,0,149.0,...,False,False,"[{'chip_name': 'bboost', 'num_played': 145393}...",28.0,225.0,279.0,"{'id': 279, 'points': 22}",16105042,283.0,28.0
4,5,Gameweek 5,2022-08-30T17:00:00Z,0,False,False,NaN,1661878800,0,NaN,...,False,False,[],NaN,NaN,NaN,None,0,NaN,NaN


Observation: events_df dataframe contains the summary of each gameweek FPL data. 

In [18]:
#explore the current season phases dataframe
data_exploration(phases_df)


 There are 11 rows and 4 columns

The columns in the dataset are
Index(['id', 'name', 'start_event', 'stop_event'], dtype='object')

The columns with null values in the dataset are
[]

The data types and null values count in the dataset are
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           11 non-null     int64 
 1   name         11 non-null     object
 2   start_event  11 non-null     int64 
 3   stop_event   11 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 480.0+ bytes
None


In [19]:
#explore the current season phases dataframe
phases_df.head()

,id,name,start_event,stop_event
0,1,Overall,1,38
1,2,August,1,5
2,3,September,6,8
3,4,October,9,14
4,5,November,15,16


Observation: phases_df dataframe lists the calendar months for each game week.

In [20]:
#explore the current season teams dataframe
data_exploration(teams_df)


 There are 20 rows and 21 columns

The columns in the dataset are
Index(['code', 'draw', 'form', 'id', 'loss', 'name', 'played', 'points',
       'position', 'short_name', 'strength', 'team_division', 'unavailable',
       'win', 'strength_overall_home', 'strength_overall_away',
       'strength_attack_home', 'strength_attack_away', 'strength_defence_home',
       'strength_defence_away', 'pulse_id'],
      dtype='object')

The columns with null values in the dataset are
['form', 'team_division']

The data types and null values count in the dataset are
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   code                   20 non-null     int64 
 1   draw                   20 non-null     int64 
 2   form                   0 non-null      object
 3   id                     20 non-null     int64 
 4   loss                   20 

In [21]:
#explore the current season teams dataframe
teams_df.head()

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1200,1270,1150,1210,1190,1220,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1090,1100,1140,1110,1090,1090,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,...,None,False,0,1050,1050,1080,1130,1060,1100,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,...,None,False,0,1090,1100,1120,1150,1080,1120,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,...,None,False,0,1100,1090,1160,1160,1100,1120,131


Observation: teams_df dataframe contains information about the performance of teams in the current season.

In [22]:
#explore the current season players performance dataframe
data_exploration(players_df)


 There are 601 rows and 67 columns

The columns in the dataset are
Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_ty

In [23]:
#explore the current season players performance dataframe
players_df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,NaN,NaN,58822,0,0,-2,2,0,2,0.5,...,356,139,432,169,5.0,,3.0,,NaN,
1,NaN,NaN,84450,0,0,0,0,1,3,6.0,...,94,60,41,30,NaN,,NaN,,NaN,
2,NaN,NaN,153256,0,0,-1,1,0,3,1.0,...,385,147,325,146,NaN,,NaN,,NaN,
3,NaN,NaN,156074,0,0,-2,2,0,2,0.7,...,358,141,434,171,NaN,,NaN,,NaN,
4,75.0,NaN,167199,0,0,0,0,0,3,1.9,...,90,57,158,85,NaN,,4.0,,NaN,


Observation: players_df dataframe contains individual performance data for players in the current season.

In [24]:
#explore the current season elements stats dataframe
data_exploration(element_stats_df)


 There are 17 rows and 2 columns

The columns in the dataset are
Index(['label', 'name'], dtype='object')

The columns with null values in the dataset are
[]

The data types and null values count in the dataset are
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   17 non-null     object
 1   name    17 non-null     object
dtypes: object(2)
memory usage: 400.0+ bytes
None


In [25]:
#explore the current season elements stats dataframe
element_stats_df.head()

,label,name
0,Minutes played,minutes
1,Goals scored,goals_scored
2,Assists,assists
3,Clean sheets,clean_sheets
4,Goals conceded,goals_conceded


Observation: elements_stats_df contains description of labels. Not required for this project.

In [26]:
#explore the current season player positions dataframe
data_exploration(element_types_df)


 There are 4 rows and 11 columns

The columns in the dataset are
Index(['id', 'plural_name', 'plural_name_short', 'singular_name',
       'singular_name_short', 'squad_select', 'squad_min_play',
       'squad_max_play', 'ui_shirt_specific', 'sub_positions_locked',
       'element_count'],
      dtype='object')

The columns with null values in the dataset are
[]

The data types and null values count in the dataset are
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    4 non-null      int64 
 1   plural_name           4 non-null      object
 2   plural_name_short     4 non-null      object
 3   singular_name         4 non-null      object
 4   singular_name_short   4 non-null      object
 5   squad_select          4 non-null      int64 
 6   squad_min_play        4 non-null      int64 
 7   squad_max_play        

In [27]:
#explore the current season player positions dataframe
element_types_df.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],61
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],215
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],260
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],65


Observation: elements_types_df dataframe contains the description of available positions for players.

In [28]:
#explore the current season fixtures dataframe
data_exploration(fixtures_df)


 There are 380 rows and 17 columns

The columns in the dataset are
Index(['code', 'event', 'finished', 'finished_provisional', 'id',
       'kickoff_time', 'minutes', 'provisional_start_time', 'started',
       'team_a', 'team_a_score', 'team_h', 'team_h_score', 'stats',
       'team_h_difficulty', 'team_a_difficulty', 'pulse_id'],
      dtype='object')

The columns with null values in the dataset are
['team_a_score', 'team_h_score']

The data types and null values count in the dataset are
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   code                    380 non-null    int64  
 1   event                   380 non-null    int64  
 2   finished                380 non-null    bool   
 3   finished_provisional    380 non-null    bool   
 4   id                      380 non-null    int64  
 5   kickoff_time           

In [29]:
#explore the current season fixtures dataframe
fixtures_df.head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,2292810,1,True,True,1,2022-08-05T19:00:00Z,90,False,True,1,2.0,7,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,2,74911
1,2292813,1,True,True,4,2022-08-06T11:30:00Z,90,False,True,12,2.0,9,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,74914
2,2292811,1,True,True,2,2022-08-06T14:00:00Z,90,False,True,2,0.0,3,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,2,74912
3,2292814,1,True,True,5,2022-08-06T14:00:00Z,90,False,True,20,1.0,11,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,74915
4,2292816,1,True,True,7,2022-08-06T14:00:00Z,90,False,True,16,0.0,15,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,3,74917


Observation: fixtures_df dataframe contains the data for all fixtures in the current season.

Data Exploration Conclusions:
The following dataframes from the keys in the dictionary to be utilised in this project are: dd_hauls, history, events, teams, elements, 

Data Cleaning and Manipulation

In [30]:
#create a feature for the year in respective season dd_hauls dataframe
dd_hauls16_df['year'] = dd_hauls16_df.apply(lambda x: "2016-17", axis=1)
dd_hauls17_df['year'] = dd_hauls17_df.apply(lambda x: "2017-18", axis=1)
dd_hauls18_df['year'] = dd_hauls18_df.apply(lambda x: "2018-19", axis=1)
dd_hauls19_df['year'] = dd_hauls19_df.apply(lambda x: "2019-20", axis=1)
dd_hauls20_df['year'] = dd_hauls20_df.apply(lambda x: "2020-21", axis=1)
dd_hauls21_df['year'] = dd_hauls21_df.apply(lambda x: "2021-22", axis=1)
dd_hauls22_df['year'] = dd_hauls22_df.apply(lambda x: "2022-23", axis=1)

In [31]:
#create a feature for players club name in the dd hauls dataframe for respective season using the history dataframe
teams=dict(zip(hist16_df.web_name, hist16_df.team_name))
dd_hauls16_df['club_name'] = dd_hauls16_df['player'].map(teams)

teams=dict(zip(hist17_df.web_name, hist17_df.team_name))
dd_hauls17_df['club_name'] = dd_hauls17_df['player'].map(teams)

teams=dict(zip(hist18_df.web_name, hist18_df.team_name))
dd_hauls18_df['club_name'] = dd_hauls18_df['player'].map(teams)

teams=dict(zip(hist19_df.web_name, hist19_df.team_name))
dd_hauls19_df['club_name'] = dd_hauls19_df['player'].map(teams)

teams=dict(zip(hist20_df.web_name, hist20_df.team_name))
dd_hauls20_df['club_name'] = dd_hauls20_df['player'].map(teams)

teams=dict(zip(hist21_df.web_name, hist21_df.team_name))
dd_hauls21_df['club_name'] = dd_hauls21_df['player'].map(teams)

teams=dict(zip(hist22_df.web_name, hist22_df.team_name))
dd_hauls22_df['club_name'] = dd_hauls22_df['player'].map(teams)
dd_hauls16_df.head()

,fixture,player,points,position,round,year,club_name
0,HUL(H),Defour,11,MID,4,2016-17,BUR
1,HUL(H),Coutinho,14,MID,6,2016-17,LIV
2,HUL(H),Lallana,15,MID,6,2016-17,LIV
3,HUL(H),Milner,14,MID,6,2016-17,LIV
4,HUL(H),Daniels,13,DEF,8,2016-17,BOU


In [32]:
#concatenate the dataframes for all the seasons
dfs = [dd_hauls16_df, dd_hauls17_df, dd_hauls18_df, dd_hauls19_df, dd_hauls20_df, dd_hauls21_df, dd_hauls22_df]
hauls_df = pd.concat(dfs, axis=0, ignore_index=True)
hauls_df.shape

(2749, 7)

In [33]:
#Create a feature linking player and opposition 
# Merge "round" and "year" into a column
hauls_df['aclub'] = hauls_df['fixture'].str.slice(0, 3)
#dd_hauls22_df['aclub'] = dd_hauls22_df['fixture'].str.slice(0, 3)
hauls_df['GW'] = hauls_df['round'].map(str) + '-' + hauls_df['year'].map(str)
#dd_hauls22_df['GW'] = dd_hauls22_df['round'].map(str) + '-' + dd_hauls22_df['year'].map(str)
hauls_df['player-opp'] = hauls_df['player'].map(str) + '-' + hauls_df['aclub'].map(str)
#dd_hauls22_df['player-opp'] = dd_hauls22_df['player'].map(str) + '-' + dd_hauls22_df['aclub'].map(str)

#Create a column for Home and Away
a='(H)'

hauls_df['Ground'] = hauls_df.fixture.apply(lambda x: "Home" if a in x else "Away")
#dd_hauls22_df['Ground'] = dd_hauls22_df.fixture.apply(lambda x: "Home" if a in x else "Away")

# dd_hauls22_df not processed yet, consider processing if data required as train data set.

In [34]:
# Quick exploration of composite data for count of hauls by players
hauls_df.groupby(['player']).size().sort_values(ascending = False).head(10)

player
Salah               49
Son                 46
Kane                46
Sterling            41
De Bruyne           41
Mané                33
Vardy               32
Alexander-Arnold    30
Mahrez              29
Hazard              26
dtype: int64

In [35]:
#Quick exploration of top 5 teams player Salah has hauls playing against.
hauls_df[hauls_df.player == 'Salah'].groupby(['fixture']).size().sort_values(ascending = False).head(5)

fixture
ARS(H)    3
WHU(A)    3
CRY(H)    3
MCI(H)    2
BHA(A)    2
dtype: int64

In [36]:
#Quick exploration of fixtures and players with highest points 
hauls_df.sort_values(by='points', ascending=False).head()

,fixture,player,points,position,round,year,club_name,aclub,GW,player-opp,Ground
656,WAT(H),Salah,29,MID,31,2017-18,LIV,WAT,31-2017-18,Salah-WAT,Home
1496,NOR(A),Antonio,26,MID,44,2019-20,WHU,NOR,44-2019-20,Antonio-NOR,Away
2281,NOR(H),Mount,24,MID,9,2021-22,CHE,NOR,9-2021-22,Mount-NOR,Home
1921,SOU(A),Son,24,MID,2,2020-21,TOT,SOU,2-2020-21,Son-SOU,Away
2215,LIV(H),Grealish,24,MID,4,2020-21,AVL,LIV,4-2020-21,Grealish-LIV,Home


In [37]:
#create feature for players club name and position played in the players_df dataframe for current season using the teams_pdf and elements_types dataframe
teams_now=dict(zip(teams_df.id, teams_df.short_name))
positions=dict(zip(element_types_df.id, element_types_df.singular_name_short))
players_df['club_name'] = players_df['team'].map(teams_now)
players_df['position'] = players_df['element_type'].map(positions)
players_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank'

In [38]:
players_df[['first_name', 'second_name', 'club_name', 'position', 'chance_of_playing_next_round', 'chance_of_playing_this_round', 'news_added', 'squad_number', 'corners_and_indirect_freekicks_order', 'direct_freekicks_order', 'penalties_order']].head(10)

,first_name,second_name,club_name,position,chance_of_playing_next_round,chance_of_playing_this_round,news_added,squad_number,corners_and_indirect_freekicks_order,direct_freekicks_order,penalties_order
0,Cédric,Alves Soares,ARS,DEF,NaN,NaN,None,None,5.0,3.0,NaN
1,Granit,Xhaka,ARS,MID,NaN,NaN,None,None,NaN,NaN,NaN
2,Mohamed,Elneny,ARS,MID,NaN,NaN,None,None,NaN,NaN,NaN
3,Rob,Holding,ARS,DEF,NaN,NaN,None,None,NaN,NaN,NaN
4,Thomas,Partey,ARS,MID,75.0,NaN,2022-08-27T20:00:06.313026Z,None,NaN,4.0,NaN
5,Martin,Ødegaard,ARS,MID,NaN,NaN,None,None,4.0,1.0,NaN
6,Kieran,Tierney,ARS,DEF,100.0,100.0,2022-08-04T15:00:05.276574Z,None,NaN,NaN,NaN
7,Nicolas,Pépé,ARS,MID,0.0,0.0,2022-08-26T07:24:29.541862Z,None,NaN,2.0,3.0
8,Benjamin,White,ARS,DEF,NaN,NaN,None,None,NaN,NaN,NaN
9,Eddie,Nketiah,ARS,FWD,NaN,NaN,None,None,NaN,NaN,NaN


In [39]:
#Cleaning the current season dataframes (null values, time series operations for dates)
players_df.chance_of_playing_next_round = players_df.chance_of_playing_next_round.fillna(100.0)
players_df.chance_of_playing_this_round = players_df.chance_of_playing_this_round.fillna(100.0)
players_df.corners_and_indirect_freekicks_order = players_df.corners_and_indirect_freekicks_order.fillna(0)
players_df.direct_freekicks_order = players_df.direct_freekicks_order.fillna(0)
players_df.penalties_order = players_df.penalties_order.fillna(0)
players_df.drop(['id', 'squad_number'], axis=1, inplace=True)
fixtures_df.team_a_score = fixtures_df.team_a_score.fillna(0)
fixtures_df.team_h_score = fixtures_df.team_h_score.fillna(0)
fixtures_df.drop(['stats', 'id'], axis=1, inplace=True)
fixtures_df['kickoff_time'] = fixtures_df['kickoff_time'].str[:-10]
fixtures_df['kickoff_time'] = pd.to_datetime(fixtures_df['kickoff_time'])

In [40]:
#Create Next Game week fixtures - This code could be used to automate GW selection.
#GW3fixtures_df = fixtures_df.loc[(fixtures_df['kickoff_time'] >= "2022-08-27") & (fixtures_df['kickoff_time'] <= "2022-08-28")].copy()
#GW3fixtures_df.head()

In [41]:
#Create Next Game week fixtures
GWfixtures_df = fixtures_df.loc[(fixtures_df['event'] == 5)].copy()
GWfixtures_df.shape

(10, 15)

In [42]:
#Mapping away and home team and their respective strengths
home_strength= dict(zip(teams_df.id, teams_df.strength_overall_home))
away_strength= dict(zip(teams_df.id, teams_df.strength_overall_away))
GWfixtures_df['away_team'] = GWfixtures_df['team_a'].map(teams_now)
GWfixtures_df['home_team'] = GWfixtures_df['team_h'].map(teams_now)
GWfixtures_df['away_team_strength'] = GWfixtures_df['team_a'].map(away_strength)
GWfixtures_df['home_team_strength'] = GWfixtures_df['team_h'].map(home_strength)
GWfixtures_df.head(10)

,code,event,finished,finished_provisional,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,team_h_difficulty,team_a_difficulty,pulse_id,away_team,home_team,away_team_strength,home_team_strength
40,2292852,5,False,False,2022-08-30,0,False,False,4,0.0,7,0.0,2,2,74953,BRE,CRY,1100,1090
41,2292853,5,False,False,2022-08-30,0,False,False,5,0.0,9,0.0,2,2,74954,BHA,FUL,1090,1040
42,2292859,5,False,False,2022-08-30,0,False,False,6,0.0,17,0.0,4,2,74960,CHE,SOU,1280,1060
43,2292854,5,False,False,2022-08-30,0,False,False,8,0.0,11,0.0,2,2,74955,EVE,LEE,1100,1070
44,2292851,5,False,False,2022-08-31,0,False,False,2,0.0,1,0.0,2,4,74951,AVL,ARS,1100,1200
45,2292850,5,False,False,2022-08-31,0,False,False,20,0.0,3,0.0,2,2,74952,WOL,BOU,1100,1050
46,2292858,5,False,False,2022-08-31,0,False,False,16,0.0,13,0.0,2,5,74959,NFO,MCI,1060,1350
47,2292856,5,False,False,2022-08-31,0,False,False,18,0.0,19,0.0,3,3,74957,TOT,WHU,1270,1110
48,2292857,5,False,False,2022-08-31,0,False,False,15,0.0,12,0.0,3,5,74958,NEW,LIV,1190,1350
49,2292855,5,False,False,2022-09-01,0,False,False,14,0.0,10,0.0,3,4,74956,MUN,LEI,1250,1110


In [43]:
#Explore current Gameweek Fixtures dataframe
data_exploration(GWfixtures_df)


 There are 10 rows and 19 columns

The columns in the dataset are
Index(['code', 'event', 'finished', 'finished_provisional', 'kickoff_time',
       'minutes', 'provisional_start_time', 'started', 'team_a',
       'team_a_score', 'team_h', 'team_h_score', 'team_h_difficulty',
       'team_a_difficulty', 'pulse_id', 'away_team', 'home_team',
       'away_team_strength', 'home_team_strength'],
      dtype='object')

The columns with null values in the dataset are
[]

The data types and null values count in the dataset are
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 40 to 49
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   code                    10 non-null     int64         
 1   event                   10 non-null     int64         
 2   finished                10 non-null     bool          
 3   finished_provisional    10 non-null     bool          
 4   kick

In [44]:
#Explore the current season players dataframe
data_exploration(players_df)


 There are 601 rows and 67 columns

The columns in the dataset are
Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'in_dreamteam', 'news',
       'news_added', 'now_cost', 'photo', 'points_per_game', 'second_name',
       'selected_by_percent', 'special', 'status', 'team', 'team_code',
       'total_points', 'transfers_in', 'transfers_in_event', 'transfers_out',
       'transfers_out_event', 'value_form', 'value_season', 'web_name',
       'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'influence_rank', 'influence_rank_type',
       'creativity_rank'

Observation: The news_added feature with null values is currently not in use. Consider addressing if required in the future.

In [45]:
players_df[['first_name', 'second_name', 'club_name', 'position', 'form', 'team']].head()

,first_name,second_name,club_name,position,form,team
0,Cédric,Alves Soares,ARS,DEF,0.0,1
1,Granit,Xhaka,ARS,MID,5.5,1
2,Mohamed,Elneny,ARS,MID,0.5,1
3,Rob,Holding,ARS,DEF,0.2,1
4,Thomas,Partey,ARS,MID,2.0,1


In [46]:
#Create features for data analysis (player-opposition team, game week fixture difficulty index, player's club )
gw_away_players = pd.merge(players_df, GWfixtures_df, how="inner", left_on=["team"], right_on=["team_a"])
gw_away_players['player_opp'] = gw_away_players['web_name'].map(str) + '-' + gw_away_players['home_team'].map(str)
gw_home_players = pd.merge(players_df, GWfixtures_df, how="inner", left_on=["team"], right_on=["team_h"])
gw_home_players['player_opp'] = gw_home_players['web_name'].map(str) + '-' + gw_home_players['away_team'].map(str)
gw_away_players['diff'] = gw_away_players['away_team_strength'] - gw_away_players['home_team_strength']
gw_home_players['diff'] = gw_home_players['home_team_strength'] - gw_home_players['away_team_strength']
players_df = gw_away_players.append(gw_home_players)
players_df['player_club'] = players_df['web_name'].map(str) + '-' + players_df['club_name'].map(str)

In [47]:
players_df[['first_name', 'second_name', 'web_name', 'club_name', 'position', 'form', 'diff', 'away_team', 'home_team', 'player_club', 'player_opp' ]].head(10)

,first_name,second_name,web_name,club_name,position,form,diff,away_team,home_team,player_club,player_opp
0,Philippe,Coutinho Correia,Coutinho,AVL,MID,1.5,-100,AVL,ARS,Coutinho-AVL,Coutinho-ARS
1,Danny,Ings,Ings,AVL,FWD,2.5,-100,AVL,ARS,Ings-AVL,Ings-ARS
2,Emiliano,Martínez Romero,Martínez,AVL,GKP,4.0,-100,AVL,ARS,Martínez-AVL,Martínez-ARS
3,Calum,Chambers,Chambers,AVL,DEF,0.8,-100,AVL,ARS,Chambers-AVL,Chambers-ARS
4,Lucas,Digne,Digne,AVL,DEF,0.8,-100,AVL,ARS,Digne-AVL,Digne-ARS
5,Bertrand,Traoré,Traoré,AVL,MID,0.0,-100,AVL,ARS,Traoré-AVL,Traoré-ARS
6,Robin,Olsen,Olsen,AVL,GKP,0.0,-100,AVL,ARS,Olsen-AVL,Olsen-ARS
7,Morgan,Sanson,Sanson,AVL,MID,0.0,-100,AVL,ARS,Sanson-AVL,Sanson-ARS
8,John,McGinn,McGinn,AVL,MID,1.8,-100,AVL,ARS,McGinn-AVL,McGinn-ARS
9,Kortney,Hause,Hause,AVL,DEF,0.0,-100,AVL,ARS,Hause-AVL,Hause-ARS


In [48]:
#Declaring my current fantasy team player selection
my_team = ['Ederson-MCI', 'Alexander-Arnold-LIV', 'Salah-LIV', 'Walker-MCI', 'Haaland-MCI', 'James-CHE', 'Schär-NEW', 'Archer-AVL', 'Rodrigo-LEE', 'Saka-ARS', 'Martinelli-ARS', 'Jesus-ARS', 'Meslier-LEE', 'Milivojevic-CRY', 'Dunk-BHA']
my_players_df = players_df.loc[players_df.player_club.isin(my_team)].copy()
my_players_df.shape

(15, 89)

In [51]:
#Declaring bank balance and identify all players currently not in my team
bank = 3
my_team = ['Ederson-MCI', 'Alexander-Arnold-LIV', 'Salah-LIV', 'Walker-MCI', 'Haaland-MCI', 'James-CHE', 'Schär-NEW', 'Archer-AVL', 'Rodrigo-LEE', 'Saka-ARS', 'Martinelli-ARS', 'Jesus-ARS', 'Meslier-LEE', 'Milivojevic-CRY', 'Dunk-BHA']
players_not_in_my_team = players_df.loc[~players_df.player_club.isin(my_team)].copy()
players_not_in_my_team.shape

(586, 89)

In [52]:
#Explore the information of my players (form, availability, next opposition, playing away or home, current cost)
my_players_df[['web_name', 'club_name', 'position', 'form', 'now_cost', 'away_team', 'home_team', 'chance_of_playing_this_round']].head(20)

,web_name,club_name,position,form,now_cost,away_team,home_team,chance_of_playing_this_round
24,Archer,AVL,FWD,0.5,45,AVL,ARS,100.0
65,Dunk,BHA,DEF,5.5,45,BHA,FUL,100.0
111,James,CHE,DEF,5.5,61,CHE,SOU,100.0
197,Schär,NEW,DEF,5.2,47,NEW,LIV,100.0
11,Saka,ARS,MID,4.0,79,AVL,ARS,100.0
17,Martinelli,ARS,MID,6.0,64,AVL,ARS,100.0
26,Jesus,ARS,FWD,6.5,82,AVL,ARS,100.0
62,Milivojevic,CRY,MID,0.8,45,BRE,CRY,100.0
149,Rodrigo,LEE,MID,9.2,64,EVE,LEE,100.0
162,Meslier,LEE,GKP,3.5,45,EVE,LEE,100.0


In [53]:
#Deciding Players to transfer out of my team (using a weighting function defined in code.py)
player_out= calc_out_weights(my_players_df)
player_out[['web_name', 'element_type', 'form', 'out_weight']]

,web_name,element_type,form,out_weight
24,Archer,4,0.5,128.333333


In [55]:
#Review the out_weight for each player in my team and decide on player to transfer  
my_players_df[['web_name', 'club_name', 'team', 'position', 'chance_of_playing_this_round', 'out_weight', 'now_cost', 'form', 'diff', 'away_team', 'home_team']].sort_values(by='out_weight', ascending=False).head(20)

,web_name,club_name,team,position,chance_of_playing_this_round,out_weight,now_cost,form,diff,away_team,home_team
24,Archer,AVL,2,FWD,100.0,128.333333,45,0.5,-100,AVL,ARS
197,Schär,NEW,15,DEF,100.0,101.333333,47,5.2,-160,NEW,LIV
62,Milivojevic,CRY,7,MID,100.0,95.333333,45,0.8,-10,BRE,CRY
162,Meslier,LEE,11,GKP,100.0,65.000000,45,3.5,-30,EVE,LEE
65,Dunk,BHA,5,DEF,100.0,28.333333,45,5.5,50,BHA,FUL
11,Saka,ARS,1,MID,100.0,26.666667,79,4.0,100,AVL,ARS
149,Rodrigo,LEE,11,MID,100.0,18.000000,64,9.2,-30,EVE,LEE
17,Martinelli,ARS,1,MID,100.0,6.666667,64,6.0,100,AVL,ARS
26,Jesus,ARS,1,FWD,100.0,1.666667,82,6.5,100,AVL,ARS
111,James,CHE,6,DEF,100.0,0.000000,61,5.5,220,CHE,SOU


In [56]:
##Deciding players In - specifying position to change, and available budget and identify invalid teams with 3 players in current team
#position = player_out.element_type.iat[0]
position = 2
#out_cost = player_out.now_cost.iat[0]
out_cost = 45
budget = bank + out_cost
dups_team = my_players_df.pivot_table(index=['club_name'], aggfunc='size')
invalid_teams = dups_team.loc[dups_team==3].index.tolist()
invalid_teams

['ARS', 'MCI']

In [57]:
potential_players=players_not_in_my_team.loc[~players_not_in_my_team.club_name.isin(invalid_teams)]
potential_players=potential_players.loc[potential_players.element_type==position]
potential_players = potential_players.loc[potential_players.now_cost<=budget]
potential_players[['web_name', 'element_type','club_name', 'position', 'chance_of_playing_this_round', 'now_cost', 'form', 'diff', 'away_team', 'home_team']].head(11)

,web_name,element_type,club_name,position,chance_of_playing_this_round,now_cost,form,diff,away_team,home_team
3,Chambers,2,AVL,DEF,100.0,44,0.8,-100,AVL,ARS
4,Digne,2,AVL,DEF,100.0,48,0.8,-100,AVL,ARS
9,Hause,2,AVL,DEF,0.0,44,0.0,-100,AVL,ARS
10,Mings,2,AVL,DEF,100.0,44,0.8,-100,AVL,ARS
14,Cash,2,AVL,DEF,100.0,48,1.2,-100,AVL,ARS
15,Konsa,2,AVL,DEF,100.0,44,1.2,-100,AVL,ARS
20,Diego Carlos,2,AVL,DEF,0.0,48,0.8,-100,AVL,ARS
26,Guilbert,2,AVL,DEF,100.0,40,0.0,-100,AVL,ARS
27,Augustinsson,2,AVL,DEF,100.0,44,0.0,-100,AVL,ARS
28,Kesler Hayden,2,AVL,DEF,0.0,40,0.0,-100,AVL,ARS


In [58]:
#Number of players available for transfer into current team
potential_players.shape

(163, 89)

In [60]:
#Confirm Invalid Teams are not included
potential_players.groupby(['club_name']).size().sort_values(ascending = False).head(20)

club_name
NFO    12
AVL    11
BRE    11
MUN    11
LEI    11
EVE    11
NEW    10
SOU     9
LIV     9
BOU     9
FUL     9
BHA     8
WOL     8
CRY     8
WHU     8
LEE     8
TOT     7
CHE     3
dtype: int64

In [ ]:
def calc_in_players_weights(players):
    players['in_weight'] = 1
    players['in_weight'] += players['diff']/3
    players['in_weight'] += players['form'].astype("float")*10
    players['in_weight'] -= (100 - players['chance_of_playing_this_round'].astype("float"))*0.2
    players.loc[players['in_weight'] <0, 'in_weight'] =0
    return players

In [ ]:
player_in = calc_in_players_weights(potential_players)
player_in[['web_name', 'club_name', 'element_type', 'chance_of_playing_this_round', 'weight', 'now_cost', 'form', 'diff', 'away_team', 'home_team']]

In [ ]:
potential_players = player_in.loc[(player_in.weight>0) & (player_in.chance_of_playing_this_round==100)] 
potential_players[['web_name', 'club_name', 'position', 'chance_of_playing_this_round', 'weight', 'now_cost', 'form', 'diff', 'away_team', 'home_team', 'player_opp']].sort_values(by='form', ascending=False).head(40)

In [ ]:
potential_players.shape

In [ ]:
#Player by random selection
potential_player_alg = potential_players.sample(1, weights=potential_players.weight)
potential_player_alg[['web_name', 'club_name', 'position', 'chance_of_playing_this_round', 'weight', 'now_cost', 'form', 'diff', 'away_team', 'home_team', 'player_opp']].head()

In [ ]:
#Checking Players with Historical Haul
potential_players2 = pd.merge(potential_players, hauls_df, how="inner", left_on=["player_opp"], right_on=["player-opp"])

In [ ]:
potential_players2[['web_name', 'player_opp', 'club_name_y', 'year']].head()

In [ ]:
def calc_starters_weights(players):
    players['weight'] = 1
    players['weight'] += players['diff']/2
    players['weight'] += players['form'].astype("float")*5
    players['weight'] -= (100 - players['chance_of_playing_this_round'].astype("float"))*0.2
    players.loc[players['weight'] <0, 'weight'] =0
    return players

In [ ]:
starters = calc_starters_weights(my_players_df)
starters[['web_name', 'club_name', 'team', 'position', 'chance_of_playing_this_round', 'weight', 'now_cost', 'form', 'diff', 'away_team', 'home_team']].sort_values(by='weight', ascending=False).head(15)